In [8]:
#Wczytanie dokumentu
import os  
import pandas as pd
import kagglehub

path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

csv_path = os.path.join(path, "IMDB Dataset.csv")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(df.head())
else:
    print(f"Plik {csv_path} nie istnieje. Sprawdź nazwę pliku i ścieżkę.")


Path to dataset files: C:\Users\LENOVO\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [16]:
#Funkcja czyszcząca do lematyzacji tekstu i usuwania interpunkcji, słów stop, białych znaków 
import spacy
nlp = spacy.load("en_core_web_sm")
def clean(text):
    doc = nlp(text)
    doc_cleaned = [
        token for token in doc 
        if not (token.is_punct or token.is_stop or token.is_space or token.is_bracket)
    ]
    doc_lemmatized = [token.lemma_ for token in doc_cleaned]
    text_lemmatized = " ".join(doc_lemmatized)
    
    return text_lemmatized


In [18]:
from tqdm import tqdm
tqdm.pandas(desc="Czyszczenie tekstów")  
df['review'] = df['review'].progress_apply(clean)
df.head()

Czyszczenie tekstów: 100%|██████████| 50000/50000 [32:29<00:00, 25.65it/s]


,review,sentiment
0,reviewer mention watch 1 Oz episode hook right...,positive
1,wonderful little production br /><br />the fil...,positive
2,think wonderful way spend time hot summer week...,positive
3,basically family little boy Jake think zombie ...,negative
4,Petter Mattei love Time money visually stunnin...,positive


In [24]:
# Wektoryzacja tekstu za pomocą TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tqdm.pandas(desc="Wektoryzacja")  
text_vectorized = tfidf_vectorizer.fit_transform(df['review'])  
x = text_vectorized
y = df['sentiment']

In [ ]:
# Wektoryzacja tekstu za pomocą CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
text_vectorized = count_vectorizer.fit_transform(df['review'])
count_df = pd.DataFrame(text_vectorized.toarray(), columns=count_vectorizer.get_feature_names_out())
x = count_df
y = df['sentiment']

In [27]:
# Wektoryzacja tekstu za pomocą HashingVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
hashing_vectorizer = HashingVectorizer(n_features=5000) 
text_vectorized = hashing_vectorizer.fit_transform(df['review'])
x = text_vectorized
y = df['sentiment']

In [28]:
#Model 1
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42,stratify=y)

# Inicjalizacja klasyfikatora Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Trenowanie modelu na zbiorze treningowym
rf_classifier.fit(x_train, y_train)

# Predykcja 
y_predicted_train = rf_classifier.predict(x_train)
y_predicted_test = rf_classifier.predict(x_test)

print("Macierz pomyłek na zbiorze treningowym:")
print(confusion_matrix(y_train, y_predicted_train))
print("Macierz pomyłek na zbiorze testowym:")
print(confusion_matrix(y_test, y_predicted_test))
print("Raport klasyfikacji na zbiorze treningowym:")
print(classification_report(y_train, y_predicted_train))
print("Raport klasyfikacji na zbiorze testowym:")
print(classification_report(y_test, y_predicted_test))

Macierz pomyłek na zbiorze treningowym:
[[12500     0]
 [    0 12500]]
Macierz pomyłek na zbiorze testowym:
[[10507  1993]
 [ 2212 10288]]
Raport klasyfikacji na zbiorze treningowym:
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     12500
    positive       1.00      1.00      1.00     12500

    accuracy                           1.00     25000
   macro avg       1.00      1.00      1.00     25000
weighted avg       1.00      1.00      1.00     25000

Raport klasyfikacji na zbiorze testowym:
              precision    recall  f1-score   support

    negative       0.83      0.84      0.83     12500
    positive       0.84      0.82      0.83     12500

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
weighted avg       0.83      0.83      0.83     25000

